<a href="https://colab.research.google.com/github/CaioCesar999/Classifica-o/blob/main/Decision_Trees_and_RandomForest_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Encontrar os melhores potenciais clientes. 

O banco Thera Bank possui uma base de clientes crescente. A maioria desses clientes são passivos (depositantes) com depósitos de tamanhos variados. O número de clientes que também são devedores (clientes de ativos) é bastante pequeno, e o banco está interessado em expandir essa base rapidamente para trazer mais negócios de crédito e, no processo, ganhar mais com os juros dos empréstimos. Em particular, a administração deseja explorar maneiras de converter seus clientes passivos em clientes de empréstimos pessoais (enquanto os mantém como depositantes). Uma campanha que o banco realizou no ano passado para clientes passivos mostrou uma taxa de conversão saudável de mais de 9% de sucesso. Isso encorajou o departamento de marketing de varejo a elaborar campanhas para melhor direcionar o marketing, a fim de aumentar a taxa de sucesso com um orçamento mínimo. O departamento quer construir um modelo de classificação que os ajude a identificar os clientes potenciais com maior probabilidade de adquirir o empréstimo. Isso aumentará a taxa de sucesso e, ao mesmo tempo, reduzirá o custo da campanha. A base de dados histórica contém informações pessoais dos clientes e informações sobre produtos contratados. O arquivo também contém um dicionário das variáveis. 

In [1]:
# Importando as bibliotecas
import pandas as pd
import numpy as np

##Leitura e Carregamento dos dados


In [3]:
df = pd.read_excel("/content/[Copy] -  Bank_Personal_Loan_Modelling.xlsx", sheet_name = "Data")

##Analise Exploratória dos dados (EDA)

In [4]:
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [6]:
df.shape

(5000, 14)

In [7]:
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [ ]:
# Através da análise das variáveis e do dicionário, algums informações relevantes:


# Variáveis categóricas:
# Personal Loan (Empréstimo Pessoal) - Este cliente aceitou o empréstimo pessoal oferecido na última campanha? Esta é a nossa variável alvo
# Securities Account (Conta de títulos) - O cliente possui conta de títulos no banco?
# CD Account (Conta de CD) - O cliente tem uma conta de certificado de depósito (CD) no banco?
# Online - O cliente usa serviços de banco pela Internet?
# Credit Card - (Cartão de crédito) - O cliente usa um cartão de crédito emitido pelo UniversalBank?

# Variáveis contínuas:

# Age (Idade) Idade do cliente
# Experience (Experiência) - anos de experiência
# Income (Renda) - Renda anual em dólares
# CCAvg (CCAvg) - gasto médio com cartão de crédito
# Mortage (Hipoteca) - Valor da hipoteca da casa

# Variáveis ​​categóricas ordinais:
# Family (Família) - tamanho da família do cliente
# Education (Educação) - nível de educação do cliente


# A variável ID não adiciona nenhuma informação interessante. 
# Não há associação entre o ID de cliente de uma pessoa e o empréstimo, também não fornece nenhuma conclusão geral para futuros clientes de empréstimo em potencial. 
# Podemos desconsiderar essas informações para o modelo.
# Zip code também pode ser descosiderada, uma vez que a informação de distnância entre zip code pode não ser realidade

In [8]:
df = df.drop(columns = ["ID","ZIP Code"])

In [9]:
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


##Divisão da base de dados entre treino e test

In [10]:
# Distruibuição da variáveis target
df["Personal Loan"].value_counts()

0    4520
1     480
Name: Personal Loan, dtype: int64

In [11]:
df.shape

(5000, 12)

In [12]:
X = df.drop(columns = ['Personal Loan'])
Y = df['Personal Loan']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

##Analise utilizando Decision tree classifier

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
clf = DecisionTreeClassifier()

In [16]:
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [17]:
y_pred = clf.predict(X_test)

In [18]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [19]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

98.3

###O modelo performou muito bem utilizando o DecisionTreeClassifier com 98.3% de acurácia.

###Analise utilizando Decision tree classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
clf = RandomForestClassifier()

In [22]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [23]:
y_pred = clf.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

98.9

###O modelo performou muito bem utilizando o RandomForestClassifier com 98.9% de acurácia superando o modelo ajustado com Arvore de decisão.